<a href="https://colab.research.google.com/github/jbinteam/010723305/blob/main/Homework9p2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  opencv-contrib-python==4.5.3.56
import numpy as np
import cv2 
import scipy
import matplotlib.pylab as plt
from skimage import io

<h1 style = "text-align: center">Custom implementation: numpy + scipy + opencv</h1>

<h1 style = "text-align: center"> Homogenous coordinate</h1>

<h2 style = "text-align: center">

$$ 
x = \left[ \begin{matrix} x\\y \end{matrix}\right] \rightarrow \overline{x} =  \left[ \begin{matrix} ax\\ay\\a \end{matrix} \right] = a \left[ \begin{matrix} x\\y\\1\end{matrix} \right]
$$

</h2>

In [ ]:
def imshow_compare(img1, img2) :
    fig, axes = plt.subplots(1,2, figsize = (20,20))
    axes[0].imshow(img1)
    axes[1].imshow(img2)

In [ ]:
def normalize(points) :
    """ Normalize points, so divide points with the last one"""
    for row in points :
        row /= points[-1]
    return points

In [ ]:
def make_homogeneous(points) :
    """Convert heterogenous to homogenous coordinate"""
    return np.vstack((points, np.ones((1,points.shape[1])))) #stacking the last coordinate with one

<h1 style = "text-align: center">
Affine matrix from point set
</h1>

<h2 style = "text-align: center">

$$
    Ma=b, a= \left( M^{T}M \right)^{-1} M^T b
$$

$$
    \left[
        \begin{matrix}
            m_{x1}&&m_{y1}&&1&&0&&0&&0\\
            0&&0&&0&&m_{x1}&&m_{y1}&&1\\
            m_{x2}&&m_{y2}&&1&&0&&0&&0\\
            0&&0&&0&&m_{x2}&&m_{y2}&&1\\
            m_{x3}&&m_{y3}&&1&&0&&0&&0\\
            0&&0&&0&&m_{x3}&&m_{y3}&&1\\
        \end{matrix}
    \right]
    \left[
        \begin{matrix}
            a_{00}\\
            a_{01}\\
            a_{02}\\
            a_{10}\\
            a_{11}\\
            a_{12}
        \end{matrix}
    \right]
    =
    \left[
        \begin{matrix}
            n_{x1}\\
            n_{y1}\\
            n_{x2}\\
            n_{y2}\\
            n_{x3}\\
            n_{y3}
        \end{matrix}
    \right]
$$

</h2>

In [ ]:
def Haffine_from_points(fp, tp) :
    """Find affine matrix tp(n) is affine transform of fp(m)"""

    if fp.shape != tp.shape :
        raise RuntimeError('Number of points do not match')

    # Preconditioning points -> normalization, standardization
    # from points (m points)
    m = np.mean(fp[:2], axis=1)
    maxstd = max(np.std(fp[:2], axis=1)) + 1e-9
    C1 = np.diag([1/maxstd, 1/maxstd, 1])
    C1[0][2] = -m[0]/maxstd
    C1[1][2] = -m[1]/maxstd
    fp_cond = np.dot(C1, fp)

    # to points (n points)
    m = np.mean(tp[:2], axis=1)
    C2 = C1.copy()
    C2[0][2] = -m[0]/maxstd
    C2[1][2] = -m[1]/maxstd
    tp_cond = np.dot(C2,tp)

    # After conditioning, points have mean zero, so translation is zero
    A = np.concatenate((fp_cond[:2], tp_cond[:2]), axis=0)
    U,S,V = np.linalg.svd(A.T) # As same as Least-square a = (M^T M)^-1 M^T b but more stability

    # Crate B and C matrices as Harley-Zisserman (2:nd ed) p 130. Multiple view computer vision
    tmp = V[:2].T
    B = tmp[:2]
    C = tmp[2:4]

    tmp2 = np.concatenate((np.dot(C, np.linalg.pinv(B)),np.zeros((2,1))), axis=1)
    H = np.vstack((tmp2, [0,0,1]))

    #decondition
    H = np.dot(np.linalg.inv(C2), np.dot(H,C1))

    #normalization and return
    return H/H[2,2]

<h1 style = "text-align: center">
Homography matrix from point set
</h1>

<h2 style = "text-align: center">

$$
    Ma=b, a= \left( M^{T}M \right)^{-1} M^T b
$$

$$
    \left[
        \begin{matrix}
            m_{x1}&&m_{y1}&&1&&0&&0&&0&&-m_{x1}n_{x1}&&-m_{y1}n_{x1}\\
            0&&0&&0&&m_{x1}&&m_{y1}&&1&&-m_{x1}n_{y1}&&-m_{y1}n_{y1}\\
            m_{x2}&&m_{y2}&&1&&0&&0&&0&&-m_{x2}n_{x2}&&-m_{y2}n_{x2}\\
            0&&0&&0&&m_{x2}&&m_{y2}&&1&&-m_{x2}n_{y2}&&-m_{y2}n_{y2}\\
            m_{x3}&&m_{y3}&&1&&0&&0&&0&&-m_{x3}n_{x3}&&-m_{y3}n_{x3}\\
            0&&0&&0&&m_{x3}&&m_{y3}&&1&&-m_{x3}n_{y3}&&-m_{y3}n_{y3}\\
            m_{x4}&&m_{y4}&&1&&0&&0&&0&&-m_{x4}n_{x4}&&-m_{y4}n_{x4}\\
            0&&0&&0&&m_{x4}&&m_{y4}&&1&&-m_{x4}n_{y4}&&-m_{y4}n_{y4}\\
        \end{matrix}
    \right]

    \left[
        \begin{matrix}
            h_{00}\\
            h_{01}\\
            h_{02}\\
            h_{10}\\
            h_{11}\\
            h_{12}\\
            h_{20}\\
            h_{21}
        \end{matrix}
    \right]
    =
    \left[
        \begin{matrix}
            n_{x1}\\
            n_{y1}\\
            n_{x2}\\
            n_{y2}\\
            n_{x3}\\
            n_{y3}\\
            n_{x4}\\
            n_{y4}
            
        \end{matrix}
    \right]
$$

after standardization

$$
    \left[
        \begin{matrix}
            -x_1&&-y_1&&-1&&0&&0&&0&&x_1x'_1&&y_1x_1&&x'_1\\
            0&&0&&0&&-x_1&&-y_1&&-1&&x_1y'_1&&y_1y'_1&&y'_1\\
            -x_2&&-y_2&&-1&&0&&0&&0&&x_2x'_2&&y_2x_2&&x'_2\\
            0&&0&&0&&-x_2&&-y_2&&-1&&x_2y'_2&&y_2y'_2&&y'_2\\
            -x_3&&-y_3&&-1&&0&&0&&0&&x_3x'_3&&y_3x_3&&x'_3\\
            0&&0&&0&&-x_3&&-y_3&&-1&&x_3y'_3&&y_3y'_3&&y'_3\\
            -x_4&&-y_4&&-1&&0&&0&&0&&x_4x'_4&&y_4x_4&&x'_4\\
            0&&0&&0&&-x_4&&-y_4&&-4&&x_4y'_4&&y_4y'_4&&y'_4\\
        \end{matrix}
    \right]
    \left[
        \begin{matrix}
            h_{00}\\
            h_{01}\\
            h_{02}\\
            h_{10}\\
            h_{11}\\
            h_{12}\\
            h_{20}\\
            h_{21}\\
            h_{22}
        \end{matrix}
    \right]
    = 0
$$

</h2>

In [ ]:
def H_from_points(fp, tp) :
    """Find homography matrix tp(n) is projective transform of fp(m)"""

    if fp.shape != tp.shape :
        raise RuntimeError('Number of points do not match')

    # Preconditioning points -> normalization, standardization
    # from points (m points)
    m = np.mean(fp[:2], axis=1)
    maxstd = max(np.std(fp[:2], axis=1)) + 1e-9
    C1 = np.diag([1/maxstd, 1/maxstd, 1])
    C1[0][2] = -m[0]/maxstd
    C1[1][2] = -m[1]/maxstd
    fp = np.dot(C1, fp)

    # to points (n points)
    m = np.mean(tp[:2], axis=1)
    maxstd = max(np.std(tp[:2], axis=1)) + 1e-9
    C2 = np.diag([1/maxstd, 1/maxstd, 1])
    C2[0][2] = -m[0]/maxstd
    C2[1][2] = -m[1]/maxstd
    tp = np.dot(C2,tp)

    # create matrix for linear method, 2 rows for each correpondence pair

    nbr_correspondences = fp.shape[1]
    A = np.zeros((2*nbr_correspondences, 9))
    for i in range(nbr_correspondences) :
        A[2*i] = [-fp[0][i], -fp[1][i], -1, 0, 0, 0, tp[0][i]*fp[0][i], tp[0][i]*fp[1][i], tp[0][i]]
        A[2*i+1] = [0, 0, 0, -fp[0][i], -fp[1][i], -1, tp[1][i]*fp[0][i], tp[1][i]*fp[1][i], tp[1][i]]

    U,S,V = np.linalg.svd(A)
    H = V[8].reshape((3,3))
    
    #decondition
    H = np.dot(np.linalg.inv(C2), np.dot(H, C1))
    
    #normalization and return
    return H/H[2,2]

In [ ]:
perspective_url = "https://raw.githubusercontent.com/jbinteam/010723305/main/images/Perspective%20image%20(1).jpg"
img = io.imread(perspective_url)
plt.imshow(img)
rows, cols = img.shape[:2]
width = 2000 # A4 Aspect ratio
height = 2828
src_points = np.float32([[314, 795], [2328, 323], [592, 3664], [2808, 3572]])
dst_points = np.float32([[314, 795], [314+width, 795], [314, 795+height] , [314+width, 795+height] ])
src_homog = make_homogeneous(src_points.T) # Transpose to convert row to column vector
dst_homog = make_homogeneous(dst_points.T) # Transpose to convert tow to column vector
print(src_points) # Heterogeneous
print(src_homog) # Homogeenous
H = H_from_points(src_homog, dst_homog)
warped_img = cv2.warpPerspective(img, H, (cols, rows))

fig, axes = plt.subplots(1,2, figsize =(10,10))
axes[0].imshow(img)
axes[1].imshow(warped_img)

<h1 style = "text-align: center">
    RANdom SAmple Consensus <a href="https://en.wikipedia.org/wiki/Random_sample_consensus">(RANSAC)</a>
</h1>
<div style="width:500px; margin:0 auto;">
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/de/Fitted_line.svg/1024px-Fitted_line.svg.png">
</div>

<h2 style = "text-align: center">
Example implementation code from <a href = https://scipy-cookbook.readthedocs.io/items/RANSAC.html?highlight=ransac> scipy ransac</a>
</h2>

In [ ]:
import numpy
import scipy # use numpy if scipy unavailable
import scipy.linalg # use numpy if scipy unavailable

## Copyright (c) 2004-2007, Andrew D. Straw. All rights reserved.

## Redistribution and use in source and binary forms, with or without
## modification, are permitted provided that the following conditions are
## met:

##     * Redistributions of source code must retain the above copyright
##       notice, this list of conditions and the following disclaimer.

##     * Redistributions in binary form must reproduce the above
##       copyright notice, this list of conditions and the following
##       disclaimer in the documentation and/or other materials provided
##       with the distribution.

##     * Neither the name of the Andrew D. Straw nor the names of its
##       contributors may be used to endorse or promote products derived
##       from this software without specific prior written permission.

## THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS
## "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT
## LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR
## A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT
## OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL,
## SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT
## LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE,
## DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY
## THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
## (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
## OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

def ransac(data,model,n,k,t,d,debug=False,return_all=False):
    """fit model parameters to data using the RANSAC algorithm
    
This implementation written from pseudocode found at
http://en.wikipedia.org/w/index.php?title=RANSAC&oldid=116358182

{{{
Given:
    data - a set of observed data points
    model - a model that can be fitted to data points
    n - the minimum number of data values required to fit the model
    k - the maximum number of iterations allowed in the algorithm
    t - a threshold value for determining when a data point fits a model
    d - the number of close data values required to assert that a model fits well to data
Return:
    bestfit - model parameters which best fit the data (or nil if no good model is found)
iterations = 0
bestfit = nil
besterr = something really large
while iterations < k {
    maybeinliers = n randomly selected values from data
    maybemodel = model parameters fitted to maybeinliers
    alsoinliers = empty set
    for every point in data not in maybeinliers {
        if point fits maybemodel with an error smaller than t
             add point to alsoinliers
    }
    if the number of elements in alsoinliers is > d {
        % this implies that we may have found a good model
        % now test how good it is
        bettermodel = model parameters fitted to all points in maybeinliers and alsoinliers
        thiserr = a measure of how well model fits these points
        if thiserr < besterr {
            bestfit = bettermodel
            besterr = thiserr
        }
    }
    increment iterations
}
return bestfit
}}}
"""
    iterations = 0
    bestfit = None
    besterr = numpy.inf
    best_inlier_idxs = None
    while iterations < k:
        maybe_idxs, test_idxs = random_partition(n,data.shape[0])
        maybeinliers = data[maybe_idxs,:]
        test_points = data[test_idxs]
        maybemodel = model.fit(maybeinliers)
        test_err = model.get_error( test_points, maybemodel)
        also_idxs = test_idxs[test_err < t] # select indices of rows with accepted points
        alsoinliers = data[also_idxs,:]
        if debug:
            print ('test_err.min()',test_err.min())
            print ('test_err.max()',test_err.max())
            print ('numpy.mean(test_err)',numpy.mean(test_err))
            print ('iteration %d:len(alsoinliers) = %d'%(
                iterations,len(alsoinliers)))
        if len(alsoinliers) > d:
            betterdata = numpy.concatenate( (maybeinliers, alsoinliers) )
            bettermodel = model.fit(betterdata)
            better_errs = model.get_error( betterdata, bettermodel)
            thiserr = numpy.mean( better_errs )
            if thiserr < besterr:
                bestfit = bettermodel
                besterr = thiserr
                best_inlier_idxs = numpy.concatenate( (maybe_idxs, also_idxs) )
        iterations+=1
    if bestfit is None:
        raise ValueError("did not meet fit acceptance criteria")
    if return_all:
        return bestfit, {'inliers':best_inlier_idxs}
    else:
        return bestfit

def random_partition(n,n_data):
    """return n random rows of data (and also the other len(data)-n rows)"""
    all_idxs = numpy.arange( n_data )
    numpy.random.shuffle(all_idxs)
    idxs1 = all_idxs[:n]
    idxs2 = all_idxs[n:]
    return idxs1, idxs2

class LinearLeastSquaresModel:
    """linear system solved using linear least squares

    This class serves as an example that fulfills the model interface
    needed by the ransac() function.
    
    """
    def __init__(self,input_columns,output_columns,debug=False):
        self.input_columns = input_columns
        self.output_columns = output_columns
        self.debug = debug
    def fit(self, data):
        A = numpy.vstack([data[:,i] for i in self.input_columns]).T
        B = numpy.vstack([data[:,i] for i in self.output_columns]).T
        x,resids,rank,s = scipy.linalg.lstsq(A,B)
        return x
    def get_error( self, data, model):
        A = numpy.vstack([data[:,i] for i in self.input_columns]).T
        B = numpy.vstack([data[:,i] for i in self.output_columns]).T
        B_fit = scipy.dot(A,model)
        err_per_point = numpy.sum((B-B_fit)**2,axis=1) # sum squared error per row
        return err_per_point
        
def test():
    # generate perfect input data

    n_samples = 500
    n_inputs = 1
    n_outputs = 1
    A_exact = 20*numpy.random.random((n_samples,n_inputs) )
    perfect_fit = 60*numpy.random.normal(size=(n_inputs,n_outputs) ) # the model
    B_exact = scipy.dot(A_exact,perfect_fit)
    assert B_exact.shape == (n_samples,n_outputs)

    # add a little gaussian noise (linear least squares alone should handle this well)
    A_noisy = A_exact + numpy.random.normal(size=A_exact.shape )
    B_noisy = B_exact + numpy.random.normal(size=B_exact.shape )

    if 1:
        # add some outliers
        n_outliers = 100
        all_idxs = numpy.arange( A_noisy.shape[0] )
        numpy.random.shuffle(all_idxs)
        outlier_idxs = all_idxs[:n_outliers]
        non_outlier_idxs = all_idxs[n_outliers:]
        A_noisy[outlier_idxs] =  20*numpy.random.random((n_outliers,n_inputs) )
        B_noisy[outlier_idxs] = 50*numpy.random.normal(size=(n_outliers,n_outputs) )

    # setup model

    all_data = numpy.hstack( (A_noisy,B_noisy) )
    input_columns = range(n_inputs) # the first columns of the array
    output_columns = [n_inputs+i for i in range(n_outputs)] # the last columns of the array
    debug = False
    model = LinearLeastSquaresModel(input_columns,output_columns,debug=debug)

    linear_fit,resids,rank,s = scipy.linalg.lstsq(all_data[:,input_columns],
                                                  all_data[:,output_columns])

    # run RANSAC algorithm
    ransac_fit, ransac_data = ransac(all_data,model,
                                     50, 1000, 7e3, 300, # misc. parameters
                                     debug=debug,return_all=True)
    if 1:
        import pylab

        sort_idxs = numpy.argsort(A_exact[:,0])
        A_col0_sorted = A_exact[sort_idxs] # maintain as rank-2 array

        if 1:
            pylab.plot( A_noisy[:,0], B_noisy[:,0], 'k.', label='data' )
            pylab.plot( A_noisy[ransac_data['inliers'],0], B_noisy[ransac_data['inliers'],0], 'bx', label='RANSAC data' )
        else:
            pylab.plot( A_noisy[non_outlier_idxs,0], B_noisy[non_outlier_idxs,0], 'k.', label='noisy data' )
            pylab.plot( A_noisy[outlier_idxs,0], B_noisy[outlier_idxs,0], 'r.', label='outlier data' )
        pylab.plot( A_col0_sorted[:,0],
                    numpy.dot(A_col0_sorted,ransac_fit)[:,0],
                    label='RANSAC fit' )
        pylab.plot( A_col0_sorted[:,0],
                    numpy.dot(A_col0_sorted,perfect_fit)[:,0],
                    label='exact system' )
        pylab.plot( A_col0_sorted[:,0],
                    numpy.dot(A_col0_sorted,linear_fit)[:,0],
                    label='linear fit' )
        pylab.legend()
        pylab.show()

if __name__=='__main__':
    pass
    # test()
    

In [ ]:
class RansacModel(object) :
    """Class for homography fit with scipy ransac"""
    def __init__(self, debug = False):
        self.debug = debug

    def fit(self, data) :
        """Homography  fitting to four selected correpondences"""
        data = data.T # Tranpose in order to convert row to column vector

        # from points (m points)
        fp = data[:3, :4]
        # to points (n points)
        tp = data[3:, :4]

        return H_from_points(fp, tp)
    
    def get_error(self, data, H) :
        """Apply homography to all correspondences, then calculate error for each point."""
        data = data.T
        
        # from points (m points)
        fp = data[:3]
        # to points (n points)
        tp = data[3:]

        #transform fp with homography
        fp_transformed = np.dot(H, fp)

        #normalized homogeneous coordinate
        for i in range(3) :
            fp_transformed[i] /= fp_transformed[2] # divide by the last row
        
        #return error per point
        error = np.sqrt(np.sum((tp-fp_transformed)**2, axis=0) ) # Euclidean distance, Reprojection error
        return error


In [ ]:
def H_from_ransac(fp, tp, model, maxiter=1000, match_threshold = 10) :
    """Homography matrix from RANSAC robust estimator"""
    data = np.vstack((fp,tp))
    H, ransac_data = ransac(data.T, model, 4, maxiter, match_threshold, 10, return_all=True)
    return H, ransac_data['inliers']

In [ ]:
left_img_url = "https://raw.githubusercontent.com/jbinteam/010723305/main/images/left-mountain.png"
right_img_url = "https://raw.githubusercontent.com/jbinteam/010723305/main/images/right-moutain.png"
left_img = io.imread(left_img_url)
right_img = io.imread(right_img_url)

left_gray = cv2.cvtColor(left_img, cv2.COLOR_BGR2GRAY)
right_gray = cv2.cvtColor(right_img, cv2.COLOR_BGR2GRAY)

imshow_compare(left_img, right_img)

<h2 style = "text-align: center">
    Sift feature detector and Brute force matcher
</h2>

<h2 style = "text-align: center">
    We need to extract feature location out from good_matches to use with H_from_ransac. <br>
    Please refer to <a href ="https://docs.opencv.org/4.5.3/d2/d29/classcv_1_1KeyPoint.html"> KeyPoint </a> and <a href = "https://docs.opencv.org/master/d4/de0/classcv_1_1DMatch.html"> DMatch </a> for opencv keypoint and descriptor data structure respectively
</h2>

In [ ]:
# Feature extraction -> feature detection, descriptor extraction
sift = cv2.SIFT_create()

left_kpts, left_desc = sift.detectAndCompute(left_gray,None)
right_kpts, right_desc = sift.detectAndCompute(right_gray,None)

# Feature descriptor matching & Distance ratio test
bf = cv2.BFMatcher()
matches = bf.knnMatch(left_desc, right_desc, k=2)

good_matches = list()

for m, n in matches :
    if m.distance < 0.7*n.distance :
        good_matches.append(m)

print(type(good_matches[0]))

m = list()
n = list()


# Prepare matched feature location data to use with H_from_ransac
for matched in good_matches :
    fp_idx = matched.queryIdx
    tp_idx = matched.trainIdx
    (xfp, yfp) = left_kpts[fp_idx].pt
    (xtp, ytp) = right_kpts[tp_idx].pt

    m.append((xfp, yfp))
    n.append((xtp, ytp))

m_np = np.asarray(m, dtype=np.float32)
n_np = np.asarray(n, dtype=np.float32)

m_homog = make_homogeneous(m_np.T)
n_homog = make_homogeneous(n_np.T)

model = RansacModel()
H, inliers = H_from_ransac(n_homog, m_homog, model) # Homography from right to left
        
rows, cols = left_img.shape[:2]
warped_img = cv2.warpPerspective(right_img,H, (2*cols, 2*rows))
imshow_compare(left_img, warped_img)


In [ ]:
warped_img[0:rows, 0:cols] = left_img
plt.figure(figsize=(20,20))
plt.imshow(warped_img)

In [ ]:
grayscale = cv2.cvtColor(warped_img, cv2.COLOR_RGB2GRAY)
mask = cv2.threshold(grayscale, 0, 255, cv2.THRESH_BINARY)[1]
plt.imshow(mask, cmap = 'gray')

# find contour inside mask 
contour, hier = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

# get contour with maximum area
max_area_contour = max(contour, key=cv2.contourArea)

# create bounding box
(x, y, w, h) = cv2.boundingRect(max_area_contour)

result = warped_img.copy()
result = result[0:h, 0:w]
plt.figure()
imshow_compare(left_img, right_img)
plt.figure(figsize=(20,20))
plt.imshow(result)

<h1 style = "text-align: center">
    Panorama: OpenCV implementation
</h1>

In [ ]:
ref_img_url = "https://raw.githubusercontent.com/jbinteam/010723305/main/images/left-mountain.png"
query_img_url = "https://raw.githubusercontent.com/jbinteam/010723305/main/images/right-moutain.png"

ref_img = io.imread(ref_img_url)
ref_gray = cv2.cvtColor(ref_img, cv2.COLOR_BGR2GRAY)

query_img = io.imread(query_img_url)
query_gray = cv2.cvtColor(query_img, cv2.COLOR_BGR2GRAY)

In [ ]:
sift = cv2.SIFT_create()

In [ ]:
ref_kpts, ref_desc = sift.detectAndCompute(ref_gray, None)
query_kpts, query_desc = sift.detectAndCompute(query_gray, None)

In [ ]:
bf = cv2.BFMatcher()
matches = bf.knnMatch(ref_desc, query_desc, k =2)

good_matches = list()
good_matches_list = list()

for m, n in matches :
    if m.distance < 0.7*n.distance :
        good_matches.append(m)
        good_matches_list.append([m])

ratio_matched_img = cv2.drawMatchesKnn(ref_img, ref_kpts, query_img, query_kpts,good_matches_list , None, flags=2)
plt.figure(figsize=(20,20))
plt.imshow(ratio_matched_img)

<h2 style = "text-align: center">
    OpenCV also has built-in function form Homography estimation with RANSAC scheme <br>
    That is <a href = "https://docs.opencv.org/4.5.3/d9/d0c/group__calib3d.html#ga4abc2ece9fab9398f2e560d53c8c9780"> findHomography() </a> which support many solving method
</h2>

In [ ]:
def crop_image(ref_img, query_img, H) : # Warp and crop
    rows, cols = ref_img.shape[:2]
    warped_img = cv2.warpPerspective(query_img, H, (2*cols, 2*rows))
    warped_img[0:rows, 0:cols] = ref_img
    grayscale = cv2.cvtColor(warped_img, cv2.COLOR_RGB2GRAY)
    mask = cv2.threshold(grayscale, 0, 255, cv2.THRESH_BINARY)[1]
    # find contour inside mask 
    contour, hier = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # get contour with maximum area
    max_area_contour = max(contour, key=cv2.contourArea)

    # create bounding box
    (x, y, w, h) = cv2.boundingRect(max_area_contour)

    result = warped_img[:h, :w]
    return result

In [ ]:
MIN_MATCH_NUMBER = 10

if len(good_matches) > MIN_MATCH_NUMBER :
    print('Enough matched feature')
    tp = np.float32([ ref_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
    fp = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)
    
    H, inlier_masks = cv2.findHomography(fp, tp, cv2.RANSAC, 10.0)
    ransac_img = cv2.drawMatchesKnn(ref_img, ref_kpts, query_img, query_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks)
    plt.figure(figsize=(20,20))
    plt.imshow(ransac_img)

    result = crop_image(ref_img, query_img, H)
    plt.figure(figsize=(20,20))
    plt.imshow(result)

else :
    print('Not enough for Homography estimation')

<h1 style = "text-align: center">
    Object detection: OpenCV Implementation
</h1>
<h2 style= "text-align: center">
    We can also use feature matching together with homography RANSAC scheme to build simple object detection as shown below:
</h>

In [ ]:
def preprocessing(img) :
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return (img, img_gray)

In [ ]:
template_img_url = "https://raw.githubusercontent.com/jbinteam/010723305/main/images/template.png"
query_img_url = "https://raw.githubusercontent.com/jbinteam/010723305/main/images/query.png"
query_img_far_url = "https://raw.githubusercontent.com/jbinteam/010723305/main/images/query-far.png"

template_img = io.imread(template_img_url)
template_img, template_gray = preprocessing(template_img)

query_img = io.imread(query_img_url)
query_img, query_gray = preprocessing(query_img)

query_far_img = io.imread(query_img_far_url)
query_far_img, query_far_gray = preprocessing(query_far_img)

imshow_compare(template_img, query_img)
imshow_compare(template_img, query_far_img)

In [ ]:
sift = cv2.SIFT_create()
bf = cv2.BFMatcher()

<h2 style= "text-align: center">
    In order to transform point with homography matrix please refer to
    <a href = "https://docs.opencv.org/4.5.3/d2/de8/group__core__array.html#gad327659ac03e5fd6894b90025e6900a7"> perspectiveTransform</a>
</h2>

In [ ]:
def feature_object_detection(template_img, template_gray, query_img, query_gray, min_match_number) :
    template_kpts, template_desc = sift.detectAndCompute(template_gray, None)
    query_kpts, query_desc = sift.detectAndCompute(query_gray, None)
    matches = bf.knnMatch(template_desc, query_desc, k=2)
    good_matches = list()
    good_matches_list = list()
    for m, n in matches :
        if m.distance < 0.7*n.distance :
            good_matches.append(m)
            good_matches_list.append([m])
    
    if len(good_matches) > min_match_number :
        src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
        dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

        H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 10.0) # H RANSAC
        # get the bounding box around template image
        h, w = template_img.shape[:2]
        template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
        transformed_box = cv2.perspectiveTransform(template_box, H)

        detected_img = cv2.polylines(query_img, [np.int32(transformed_box)], True, (255,0,0), 3, cv2.LINE_AA)
        drawmatch_img = cv2.drawMatchesKnn(template_img, template_kpts, detected_img, query_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks)

        return detected_img, drawmatch_img
    else :
        print('Keypoints not enough')
        return
    

In [ ]:
detected, drawmatch =  feature_object_detection(template_img, template_gray, query_img, query_gray, 10)
plt.figure(figsize=(20,20))
plt.imshow(detected)
plt.figure(figsize=(20,20))
plt.imshow(drawmatch)

In [ ]:
detected, drawmatch =  feature_object_detection(template_img, template_gray, query_far_img, query_far_gray, 10)
plt.figure(figsize=(20,20))
plt.imshow(detected)
plt.figure(figsize=(20,20))
plt.imshow(drawmatch)

<h1 style="text-align: center"> Image stitching (Panorama) exercise</h1>

<h2>แบบฝึกหัดที่ 9.2</h2>
<h4>วัตถุประสงค์ </h1>

- ทักษะการทำ image stitching
<h4>โจทย์</h4>

- ให้นักศึกษาเขียน code ซอฟต์แวร์นำข้อมูลภาพถ่ายทางอากาศเพื่อการเกษตรนำมาต่อเข้าด้วยกันให้เป็นภาพใหญ่เพียงภาพเดียวด้วยเทคนิด image stitching
- ในการนำภาพมาเย็บต่อกันให้ใช้ฟังก์ชัน stitch_and_crop ที่ได้เตรียมไว้ด้านล่าง พร้อมอธิบายการทำงานฟังก์ชันดังกล่าวลงใน google colab
- credit <a href="https://blog.samaritakis.gr/en/quick-photo-stitching-using-7-aerial-photos-vineyards/">https://blog.samaritakis.gr/en/quick-photo-stitching-using-7-aerial-photos-vineyards/  </a>

- ชุดข้อมูลภาพอยู่ใน <a href = "https://github.com/jbinteam/010723305/tree/main/images">https://github.com/jbinteam/010723305/tree/main/images</a><br>
<a href = "https://raw.githubusercontent.com/jbinteam/010723305/main/images/IMG_2010.jpg">https://raw.githubusercontent.com/jbinteam/010723305/main/images/IMG_2010.jpg</a><br>
<a href = "https://raw.githubusercontent.com/jbinteam/010723305/main/images/IMG_2011.jpg">https://raw.githubusercontent.com/jbinteam/010723305/main/images/IMG_2011.jpg</a><br>
<a href = "https://raw.githubusercontent.com/jbinteam/010723305/main/images/IMG_2012.jpg">https://raw.githubusercontent.com/jbinteam/010723305/main/images/IMG_2012.jpg</a><br>
<a href = "https://raw.githubusercontent.com/jbinteam/010723305/main/images/IMG_2013.jpg">https://raw.githubusercontent.com/jbinteam/010723305/main/images/IMG_2013.jpg</a><br>
<a href = "https://raw.githubusercontent.com/jbinteam/010723305/main/images/IMG_2014.jpg">https://raw.githubusercontent.com/jbinteam/010723305/main/images/IMG_2014.jpg</a><br>
<a href = "https://raw.githubusercontent.com/jbinteam/010723305/main/images/IMG_2015.jpg">https://raw.githubusercontent.com/jbinteam/010723305/main/images/IMG_2015.jpg</a><br>
<a href = "https://raw.githubusercontent.com/jbinteam/010723305/main/images/IMG_2016.jpg">https://raw.githubusercontent.com/jbinteam/010723305/main/images/IMG_2016.jpg</a><br>
- ผลลัพธ์ที่คาดหวัง

<img src = "https://raw.githubusercontent.com/jbinteam/010723305/main/images/Stitched.png" >

In [ ]:
def stitch_and_crop(ref_img, query_img, H) : # Warp and stitch
    rows, cols = ref_img.shape[:2]
    warped_img = cv2.warpPerspective(query_img, H, (2*cols, 2*rows))
    enlarge_ref_img = np.zeros((2*rows, 2*cols, 3), dtype=np.uint8)
    enlarge_ref_img[:rows, :cols] = ref_img
    warped_gray = cv2.cvtColor(warped_img, cv2.COLOR_RGB2GRAY)
    warped_mask = cv2.threshold(warped_gray, 0, 255, cv2.THRESH_BINARY)[1]
    segmented_ref_img = cv2.add(warped_img, enlarge_ref_img, mask=np.bitwise_not(warped_mask))
    result = cv2.add(segmented_ref_img, warped_img)

    grayscale = cv2.cvtColor(result, cv2.COLOR_RGB2GRAY)
    mask = cv2.threshold(grayscale, 0, 255, cv2.THRESH_BINARY)[1]
    # # find contour inside mask 
    contour, hier = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # # get contour with maximum area
    max_area_contour = max(contour, key=cv2.contourArea)

    # # create bounding box
    (x, y, w, h) = cv2.boundingRect(max_area_contour)

    return result[:h, :w]

In [ ]:
img_list = list()
for i in range(7) :

    img = io.imread('https://raw.githubusercontent.com/jbinteam/010723305/main/images/IMG_201{}.jpg'.format(i))
    img_list.append(img)

def image_stitching(img_list) :
    
    ## coding here ##
    
    result = stitch_and_crop(ref_img, query_img, H)

    return result

result = image_stitching(img_list)
plt.imshow(result)